In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

In [3]:
sys.path.insert(0, '/ssd/scratch/khugueni/librascal-pypi-release/build')

In [4]:
%matplotlib inline
from matplotlib import pylab as plt

import time
import rascal

import ase
from ase import Atoms
from ase.io import read, write
from ase.build import make_supercell
from ase.visualize import view
from numpy.testing import assert_allclose
import numpy as np
from numpy.linalg import norm

from scipy.integrate import quad
from pylode.lib.radial_basis import innerprod
from scipy.special import gamma, hyp1f1, sph_harm, iv

# Descriptor related imports: compare the librascal and pyLODE versions of SOAP
from rascal.representations import SphericalExpansion as RascalSphericalExpansion
#from librascal.representations import SphericalExpansion as RascalSphericalExpansion
import rascaline
from pylode.lib.projection_coeffs import DensityProjectionCalculator

# Introduction

This notebook is used to compare the features obtained from rascaline, librascal and pylode. The goal is to fully understand the details of the three implementations to make sure that the same coefficients from different codes agree.

## Table of Contents:
* [Preparation](#preparation)
    * [Exact Expression for Coefficients](#exactexpressions)
    * [Get the features for dimers](#getfeatures)
* [Coefficients for $l=1$](#l=1)
    * [Explicit computation of coefficients for $l=1$](#l=1explicit)
    * [Comparison to pyLODE/rascal for dimers](#l=1dimer)
* [Coefficients for $l\geq 2$](#lgeq2)
    * [Coefficients for $l=2$](#l=2)
    * [Coefficients for general $l$](#l_general)
* [Coefficients for l=0](#l=0)
    * [Exact coefficients](#l=0exact)
    * [Comparison to pyLODE/rascal for center contribution](#l=0center)
    * [Comparison to pyLODE/rascal for dimers](#l=0dimer)

# Preparation: Get features from all three codes <a class="anchor" id="preparation"></a>

## Exact Expression for Coefficients <a class="anchor" id="exactexpressions"></a>

### Define common hyperparameters

In [5]:
nmax = 5
lmax = 1
rcut = 6.
smearing = 0.5

### Generate orthonormalization matrix <a class="anchor" id="orthonormalization"></a>

In [6]:
Nradial = 1000
sigma = np.ones(nmax, dtype=float)
for i in range(1, nmax):
    sigma[i] = np.sqrt(i)
sigma *= rcut / nmax

# Define primitive GTO-like radial basis functions
f_gto = lambda n, x: x**n * np.exp(-0.5 * (x / sigma[n])**2)
xx = np.linspace(0, rcut * 2.5, Nradial)
R_n = np.array([f_gto(n, xx) for n in range(nmax)])

In [7]:
# Define inner product matrix using analytical expression for primitive GTOs
innerprods_primitive = np.zeros((nmax, nmax))
for n1 in range(nmax):
    for n2 in range(nmax):
        sigeff = 0.5 * (1/sigma[n1]**2 + 1/sigma[n2]**2)
        neff = 0.5 * (3 + n1 + n2)
        innerprods_primitive[n1, n2] = 0.5 * sigeff**(-neff) * gamma(neff)

eigvals, eigvecs = np.linalg.eigh(innerprods_primitive)
transformation_primitive = eigvecs @ np.diag(np.sqrt(1. / eigvals)) @ eigvecs.T

In [8]:
normalizations = np.zeros((nmax,))
for n in range(nmax):
    normalizations[n] = 1/np.sqrt(innerprods_primitive[n, n])

In [9]:
# Inner product matrix for normalized GTOs
innerprods_normalized = np.zeros((nmax, nmax))
for n1 in range(nmax):
    for n2 in range(nmax):
        innerprods_normalized[n1, n2] = innerprods_primitive[n1, n2]
        innerprods_normalized[n1, n2] *= normalizations[n1] * normalizations[n2]

In [10]:
eigvals, eigvecs = np.linalg.eigh(innerprods_normalized)
transformation_normalized = eigvecs @ np.diag(np.sqrt(1. / eigvals)) @ eigvecs.T

In [11]:
use_gto_convention = 'normalized' # choose between 'normalized' and 'primitive'
if use_gto_convention == 'normalized':
    transformation = transformation_normalized
    innerprods = innerprods_normalized
elif use_gto_convention == 'primitive':
    transformation = transformation_primitive
    innerprods = innerprods_primitive
    normalizations = np.ones_like(normalizations)

In [12]:
assert_allclose(transformation @ innerprods @ transformation, np.eye(nmax), atol=1e-5, rtol=1e-5)

### Fully analytical formula for general $l$

In [13]:
def coefficients_analytical_general_l(nmax, d, l):
    # Define auxilary quantities and prefactors
    a = 1. / (2 * smearing**2)
    lplus3half = l + 1.5
    prefac_global = np.pi**1.5 * sph_harm(0, l, 0, 0).real * a**l / gamma(lplus3half)
    prefac_global *= d**l * np.exp(-a*d**2)
    prefac_global /= (np.pi * smearing**2)**(3/4)

    # Start main loop
    featvec = np.zeros((nmax))
    for n in range(nmax):
        nlplus3half = (3 + n + l) / 2
        b = 1. / (2 * sigma[n]**2)
        prefac_n_dep = gamma(nlplus3half) / (a+b)**nlplus3half
        hyp = hyp1f1(nlplus3half, lplus3half, a**2*d**2/(a+b))
        featvec[n] = normalizations[n] * prefac_n_dep * hyp

    #return prefac_global * featvec
    return prefac_global * (transformation @ featvec)   

### Semianalytical (final integral performed numerically) formula for general l

In [14]:
def coefficients_semianalytical_general_l(nmax, d, l):
    # Prefactor
    a = 1. / (2 * smearing**2)
    prefac_global = 4*np.pi * sph_harm(0, l, 0, 0).real * np.exp(-a*d**2)
    prefac_global /= (np.pi * smearing**2)**(3/4)

    # Main loop performing the numerical integration
    featvec = np.zeros((nmax))
    for n in range(nmax):
        # Define the integrand consisting of a power-law, Gaussian and Bessel part
        b = 1. / (2 * sigma[n]**2)
        gaussian = lambda r: np.exp(-(a+b)*r**2)
        power = lambda r: r**(2+n)
        mod_sph_bessel = lambda x: np.sqrt(np.pi/2/x) * iv(l + 0.5, x)
        bessel = lambda r: mod_sph_bessel(2*a*d*r)
        integrand = lambda r: power(r) * gaussian(r) * bessel(r)
        
        # Numerical integration
        eps = 1e-10
        featvec[n] = normalizations[n] * quad(integrand, eps, 4.5*rcut)[0]

    #return prefac_global * featvec
    return prefac_global * (transformation @ featvec)   

## Get the features for dimers <a class="anchor" id="getfeatures"></a>

### Define structure to be used for the comparison
The main differences between librascal and the pyLODE implementation are the presence of a smooth cutoff function and the potentially different order in which neighbors are stored. We thus wish to generate structures for which these two effects do not alter the coefficients too much. For this task, we use clusters of Oxygen atoms for which all atoms have a mutual distance of less than 3A. Then, even for a relatively large smearing of 1.5A, the atomic densities will be reasonably contained within a ball of cutoff radius 6 that will also be used for the cutoff.

In [15]:
frames = []
cell = np.eye(3) * 16
Ndimers = 15
distances = np.linspace(1., 2.5, Ndimers)
for d in distances:
    positions = [[1,1,1],[1,1,1+d]]
    frame = Atoms('O2', positions=positions, cell=cell, pbc=True)
    frames.append(frame)

### Get the features from pyLODE

In [16]:
hypers = {
    'smearing':smearing,
    'max_angular':lmax,
    'max_radial':nmax,
    'cutoff_radius':rcut,
    'potential_exponent':0,
    'radial_basis': 'gto',
    'compute_gradients':False,
    'subtract_center_contribution':False,
    }

calculator_pylode = DensityProjectionCalculator(**hypers)
calculator_pylode.transform(frames)
features_pylode = calculator_pylode.features

### Get the features from librascal

In [23]:
# define the parameters of the spherical expansion
hypers = dict(interaction_cutoff=rcut, 
              max_radial=nmax, 
              max_angular=lmax, 
              gaussian_sigma_constant=smearing,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.1,
              radial_basis="GTO",
              compute_gradients=False,
              expansion_by_species_method='structure wise',
              optimization=dict(Spline=dict(accuracy=1e-12)),
              )

calculator_librascal = RascalSphericalExpansion(**hypers)
# compute the representation of all the structures
features_librascal = calculator_librascal.transform(frames).get_features(calculator_librascal)

### Get the features from Rascaline

In [24]:
HYPER_PARAMETERS = {
    "cutoff": rcut,
    "max_radial": nmax,
    "max_angular": lmax,
    "atomic_gaussian_width": smearing,
    "gradients": False,
    "center_atom_weight": 1.0,
    "radial_basis": {
        "Gto": {},
    },
    "cutoff_function": {
        "Step": {"width": 1e-3},
    },
}

calculator_rascaline = rascaline.SphericalExpansion(**HYPER_PARAMETERS)

# run the actual calculation
descriptor_rascaline = calculator_rascaline.compute(frames)

# Compare coefficients for $l=1$ <a class="anchor" id="l=1"></a>

## Explicit computation of coefficients for $l=1$ <a class="anchor" id="l=1explicit"></a>

To make sure that we understand every single aspect of the codes including all the prefactors and conventions, we compute the exact coefficients we would expect from an Oxygen molecule oriented along the z-axis. The computations are done using three (four) different methods: a fully analytical approach applicable to all $l$, a semi-analytical approach in which the final radial integral is evaluated numerically also valid for all $l$, and finally an explicit formula that was obtained directly for the special case $l=1$ starting at the very definition. For convenience, we also include the complete analytical formula for the special case $l=1$.

### Fully analytical formula for $l=1$

In [28]:
def coefficients_analytical_l1(nmax, d):
    # Define auxilary quantities and prefactors
    a = 1. / (2 * smearing**2)
    lplus3half = 2.5
    prefac_global = np.pi * np.sqrt(3/4) * a / gamma(lplus3half) * d * np.exp(-a*d**2)
    prefac_global /= (np.pi*smearing**2)**(3/4)
    
    # Start main loop
    featvec = np.zeros((nmax))
    for n in range(nmax):
        # Compute contribution for primitive GTO function
        nlplus3half = (4 + n) / 2
        b = 1. / (2 * sigma[n]**2)
        prefac_n_dep = gamma(nlplus3half) / (a+b)**nlplus3half
        hyp = hyp1f1(nlplus3half, lplus3half, a**2*d**2/(a+b))
        featvec[n] = normalizations[n] * prefac_n_dep * hyp

    return prefac_global * (transformation @ featvec)

### Semianalytical (final integral performed numerically) formula for $l=1$

In [29]:
def coefficients_semianalytical_l1(nmax, d):
    # Prefactor
    a = 1. / (2 * smearing**2)
    prefac = np.sqrt(1.5) / (2 * np.pi * smearing**3)
    prefac *= np.exp(-a*d**2)
    prefac *= (2*np.pi*smearing**2)**1.5 / (np.pi*smearing**2)**(3/4)
    
    # Start main loop
    featvec = np.zeros((nmax,))
    for n in range(nmax):
        # Start defining functions appearing in integrand
        b = 1. / (2 * sigma[n]**2)
        gaussian = lambda r: np.exp(-(a+b)*r**2)
        power = lambda r: r**(2+n)
        mod_sph_bessel = lambda x: (x*np.cosh(x) - np.sinh(x))/x**2
        bessel = lambda r: mod_sph_bessel(2*a*d*r)

        integrand = lambda r: power(r) * gaussian(r) * bessel(r) * 2
        
        # Numerical Integration
        featvec[n] = normalizations[n] * quad(integrand, 1e-10, 4.5 * rcut)[0]
    
    return prefac * (transformation @ featvec)

### Compare the coefficients obtained using different methods

In [30]:
features_analytical_l1 = np.zeros((len(distances), nmax))
features_semianalytical_l1 = np.zeros((len(distances), nmax))
features_analytical_general = np.zeros((len(distances), nmax))
features_semianalytical_general = np.zeros((len(distances), nmax))

for i_dist, d in enumerate(distances):
    features_analytical_l1[i_dist] = coefficients_analytical_l1(nmax, d)
    features_semianalytical_l1[i_dist] = coefficients_semianalytical_l1(nmax, d)
    features_analytical_general[i_dist] = coefficients_analytical_general_l(nmax, d, l=1)
    features_semianalytical_general[i_dist] = coefficients_semianalytical_general_l(nmax, d, l=1)

In [31]:
delta1 = np.linalg.norm(features_analytical_l1 - features_analytical_general)
delta2 = np.linalg.norm(features_analytical_general - features_semianalytical_general)
delta3 = np.linalg.norm(features_semianalytical_l1 - features_semianalytical_general)
print(delta1)
print(delta2)
print(delta3)

1.212138028838794e-16
2.548252555056503e-15
1.3430436943151222e-15


## Compare the exact values with those obtained from the three codes <a class="anchor" id="l=1dimer"></a>

The exact coefficients obtained using the analytical formula are now compared to those obtained from the three codes.

### Get the $l=1$ (and $m=0$) part of the features of the three codes

In [32]:
print('Shapes of feature vectors')
print('rascaline', descriptor_rascaline.values.shape)
print('librascal', features_librascal.shape)

Shapes of feature vectors
rascaline (30, 20)
librascal (30, 20)


In [33]:
features_L1_rascaline = descriptor_rascaline.values[0::2, 2*nmax:3*nmax] / 10.962374348347298 

In [34]:
features_L1_librascal = features_librascal[0::2, 2::(lmax+1)**2]

In [35]:
features_L1_pylode = features_pylode[0::2, 0, :, 2]

In [36]:
for i,features in enumerate([features_L1_rascaline, features_L1_librascal, features_L1_pylode, features_analytical_l1]):
    assert features.shape == (len(distances), nmax)

In [37]:
print(features_analytical_l1 / features_L1_pylode)

[[ 8.46699747e-01  6.72185022e+00 -1.83005355e-01  3.39258157e+00
  -1.67284502e-01]
 [ 8.29632652e-01  4.64659278e+00 -1.57082937e-01  1.08742771e+00
  -1.81754301e-01]
 [ 8.08371689e-01  3.97597631e+00 -1.22381295e-01  6.17723155e-01
  -1.73662280e-01]
 [ 7.82183913e-01  3.71430288e+00 -8.02365228e-02  3.99369829e-01
  -1.50056768e-01]
 [ 7.50342069e-01  3.64673434e+00 -3.21383405e-02  2.67435989e-01
  -1.18419736e-01]
 [ 7.12208121e-01  3.70846399e+00  2.05398877e-02  1.77935704e-01
  -8.49752546e-02]
 [ 6.67357919e-01  3.88643303e+00  7.67223626e-02  1.14062038e-01
  -5.39720266e-02]
 [ 6.15748513e-01  4.19965700e+00  1.35728523e-01  6.78169444e-02
  -2.77398323e-02]
 [ 5.57914841e-01  4.70651464e+00  1.97313942e-01  3.47148811e-02
  -7.13120447e-03]
 [ 4.95160525e-01  5.54514044e+00  2.61687825e-01  1.18920952e-02
   7.96259136e-03]
 [ 4.29685019e-01  7.07833923e+00  3.29539336e-01 -2.66331774e-03
   1.81798460e-02]
 [ 3.64580699e-01  1.05944931e+01  4.02103700e-01 -1.04541576e-02

In [38]:
print(features_analytical_l1 / features_L1_rascaline)

[[73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]
 [73.1664572 73.1664572 73.1664572 73.1664572 73.1664572]]


In [39]:
print(features_semianalytical_general / features_L1_librascal)

[[26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]
 [26.69730293 26.69730293 26.69730293 26.69730293 26.69730293]]


In [ ]:
for i, d in enumerate(distances):
    print('Distance ', i)
    coeffs_rascaline = descriptor_rascaline.values[2*i].reshape(((lmax+1)**2, nmax)).T
    coeffs_librascal = features_librascal[2*i].reshape((nmax,(lmax+1)**2))
    coeffs_pylode = features_pylode[2*i,0]
    
    for coeffs in [coeffs_rascaline, coeffs_librascal, coeffs_pylode]:
        print(coeffs[:,2] / features_analytical_general[i])
    
    print()

# Compare coefficients for $l\geq 2$ <a class="anchor" id="lgeq2"></a>

## Compare coefficients for $l=2$ <a class="anchor" id="l=2"></a>

In [40]:
lm_index = 6 # l**2 + l

In [41]:
print('Shapes of feature vectors')
print('rascaline', descriptor_rascaline.values.shape)
print('librascal', features_librascal.shape)

Shapes of feature vectors
rascaline (30, 20)
librascal (30, 20)


In [42]:
features_L2_rascaline = descriptor_rascaline.values[0::2, lm_index*nmax:(lm_index+1)*nmax]

In [43]:
features_L2_librascal = features_librascal[0::2, lm_index::(lmax+1)**2]

In [44]:
features_L2_pylode = features_pylode[0::2, 0, :, lm_index]

IndexError: index 6 is out of bounds for axis 3 with size 4

In [ ]:
features_analytical_l2 = np.zeros((len(distances), nmax))
features_numerical_l2 = np.zeros((len(distances), nmax))

for i_dist, d in enumerate(distances):
    features_analytical_l2[i_dist] = coefficients_analytical_general_l(nmax, d, l=2)
    features_numerical_l2[i_dist] = coefficients_semianalytical_general_l(nmax, d, l=2)

In [ ]:
delta_l2 = np.linalg.norm(features_analytical_l2 - features_numerical_l2)

In [ ]:
print(features_L2_pylode / features_analytical_l2)

In [ ]:
print(features_L2_pylode / features_numerical_l2)

In [ ]:
print(features_L2_librascal / features_numerical_l2)

## Compare coefficients for general $l$ <a class="anchor" id="l_general"></a>

In [ ]:
for l in range(1, lmax+1):
    lm_index = l**2 + l
    
    # Get the features from the spherical expansion codes
    features_l_pylode = features_pylode[0::2, 0, :, lm_index]
    features_l_rascaline = descriptor_rascaline.values[0::2, lm_index*nmax:(lm_index+1)*nmax] / 10.962374348347298 
    features_l_librascal = features_librascal[0::2, lm_index::(lmax+1)**2]
    
    # Get the features from the analytical and numerical evaluations
    features_l_analytical = np.zeros((len(distances), nmax))
    features_l_numerical = np.zeros((len(distances), nmax))
    for i_dist, d in enumerate(distances):
        features_l_analytical[i_dist] = coefficients_analytical_general_l(nmax, d, l=l)
        features_l_numerical[i_dist] = coefficients_semianalytical_general_l(nmax, d, l=l)
    assert_allclose(features_l_analytical, features_l_numerical, atol=1e-9, rtol=1e-7)
    
    # Compare the coefficients
    ratios = features_l_pylode / features_l_numerical
    err = np.linalg.norm(features_l_analytical - features_l_numerical)
    print(f'pyLODE vs numerical coefficients for l = {l}')
    print(np.round(ratios,6))
    print('Difference between numerical and analytical evaluation = ', err)
    print()

# Coefficients for $l=0$  <a class="anchor" id="l=0"></a>

## Compute the exact expressions for reference <a class="anchor" id="l=0exact"></a>

### Hybrid analytical-numerical computation

Here, we use the convention that the Gaussian density is normalized to one, i.e. contains the factor of $1/(2\pi\sigma^2)^{3/2}$. The spherical harmonic $Y_{00}$ is the constant function whose value is $1/\sqrt{4\pi}$ and the GTO's are orthonormalized starting from the primitive ones using the usual procedure.

In [ ]:
def compute_coeffs_explicit_l0(d):
    # Start computing coefficients for l=0
    featvec = np.zeros((nmax))
    
    # Case 1: If the distance is zero, the integrals
    # can easily be computed by hand. Furthermore,
    # separating this case out is convenient since
    # the general expression as a function of the pair
    # distance d has a (removable) singularity at d=0
    if d == 0:
        prefac = np.sqrt(np.pi)
        prefac /= np.sqrt(2 * np.pi * smearing**2)**3
        
        featvec = np.zeros((nmax))
        for n in range(nmax):
            b = 0.5 * (1/smearing**2 + 1/sigma[n]**2)
            neff = (3+n) / 2
            featvec[n] = prefac * b**(-neff) * gamma(neff)
    
    # Case 2: For nonzero pair distances (i.e. all proper neighbor contributions),
    # the final integral over the radial variable results in a hypergeometric function
    # in a fully analytical treatment. Here, we perform the final integral numerically.
    else:
        # Define auxilary quantities and prefactors
        a = d / smearing**2
        prefac = 1. / (np.sqrt(2) * np.pi * smearing * d) * np.exp(-0.5*d**2/smearing**2)

        # Start main loop
        for n in range(nmax):
            # Compute contribution for primitive GTO function
            powerlaw = lambda r: r**(1+n)
            b = 0.5 * (1/smearing**2 + 1/sigma[n]**2)
            gaussian = lambda r: np.exp(-b * r**2)
            sinh = lambda r: np.sinh(a * r)
            integrand = lambda r: powerlaw(r) * gaussian(r) * sinh(r)

            # Numerical Integration
            featvec[n] = prefac * quad(integrand, 1e-10, 4.5 * rcut)[0]
    #return featvec
    return transformation @ featvec

In [ ]:
# Start plotting the behavior of the neighbor contribution as a function of the pair distance r_ij
Ndist = 50
dd = np.linspace(0, rcut, Ndist)
coeffs_exact = np.zeros((nmax, Ndist))
for i, d in enumerate(dd):
    coeffs_exact[:,i] = compute_coeffs_explicit_l0(d)


# Make plot
for n in range(nmax):
    plt.plot(dd, coeffs_exact[n], label=f'n={n}')
plt.legend()
plt.xlabel('Pair distance rij')
plt.ylabel('Contribution to <n00|rho_i>')

Remark: This plot was mostly used to make sure that the implementation for small distances $r_{ij} \rightarrow 0$ does indeed converge to the special case $r_{ij}=0$ which is implemented separately.

### Comparison to fully analytical general expression

In [ ]:
coeffs_from_general_analytical = np.zeros((nmax, Ndist))
coeffs_from_general_numerical = np.zeros((nmax, Ndist))
for i, d in enumerate(dd):
    coeffs_from_general_analytical[:,i] = coefficients_analytical_general_l(nmax, d, l=0)
    coeffs_from_general_numerical[:,i] = coefficients_semianalytical_general_l(nmax, d, l=0)

In [ ]:
# Make plot
for n in range(nmax):
    plt.plot(dd, coeffs_exact[n], 'k')
    plt.plot(dd, coeffs_from_general_analytical[n], 'b')
    plt.plot(dd, 0.005+coeffs_from_general_numerical[n], 'r')
    
plt.plot(dd, coeffs_exact[0], label=f'exact', c='k')
plt.plot(dd, 0.01+coeffs_from_general_analytical[0], label=f'analytical', c='b')
plt.plot(dd, -0.01+coeffs_from_general_numerical[0], label=f'numerical', c='r')
plt.legend()
plt.xlabel('Pair distance rij')
plt.ylabel('Contribution to <n00|rho_i>')

In [ ]:
for n in range(nmax):
    diff1 = coeffs_exact[n] - coeffs_from_general_analytical[n]
    diff2 = coeffs_from_general_numerical[n] - coeffs_from_general_analytical[n]
    print(np.linalg.norm(diff1))
    print(np.linalg.norm(diff2[1:])) # first entry at d=0 is NaN for numerical evaluation

## Compare exact coefficients with those from pyLODE and Rascal: Center atom contribution <a class="anchor" id="l=0center"></a>

Note that $l=0$ is more complicated compared to $l=1,2,\dots$ even for dimers since the center atom contributes to the density as well. We begin this investigation by using a frame that only consists of a single atom.

### Get the coefficients from the three codes

In [ ]:
frames.append(Atoms('O', positions=[[1,1,1]], cell=cell, pbc=True))
frames.append(Atoms('O', positions=[[1,1,1]], cell=1.2*cell, pbc=True))

In [ ]:
hypers = {
    'smearing':smearing,
    'max_angular':lmax,
    'max_radial':nmax-2,
    'cutoff_radius':rcut,
    'potential_exponent':0,
    'radial_basis': 'gto',
    'compute_gradients':False,
    'subtract_center_contribution':False,
    }

calculator_pylode = DensityProjectionCalculator(**hypers)

In [ ]:
features_librascal_single = calculator_librascal.transform(frames).get_features(calculator_librascal)[-2:]
calculator_pylode.transform(frames)
features_pylode_single = calculator_pylode.features[-2:]
features_rascaline_single = calculator_rascaline.compute(frames).values[-2:]

### Sanity check 1: Make sure the feature vectors only contain nonzero entries for l=0

In [ ]:
features_pylode_single.shape

In [ ]:
features_librascal_single.shape

In [ ]:
features_rascaline_single.shape

In [ ]:
features_pylode_single

In [ ]:
print(features_rascaline_single)

Comment: Rascaline seems to exclude the center contribution by default, leading to coefficients that are zero everywhere. This is not an error of the code per se, but is something that should be kept in mind.

Make sure that the coefficients apart from those at $l=0$ are zero (since we only have a single atom in the structure, the density is spherically symmetric and thus only the $l=0$ projection survives)

In [ ]:
for i in range(len(features_pylode_single)):
    # The l=0 coefficients for pyLODE are contained in [n,l]=[:,0]
    feat = features_pylode_single[i,0]
    feat_zero = feat[:,1:]
    assert_allclose(feat_zero, np.zeros_like(feat_zero), atol=3e-6)
    
    # The l=0 coefficients for librascal are separated by (lmax+1)**2
    feat = features_librascal_single[i]
    for n in range(nmax):
        for lm in range((lmax+1)**2):
            if lm != 0:
                assert abs(feat[n*(lmax+1)**2 + lm]) < 1e-8
    
    # The l=0 coefficients of rascaline are the first nmax entries.
    # Thus, all the remaining contributions should be zero.
    #feat = np.round(features_rascaline_single[i],15)
    #assert_allclose(feat[nmax:], np.zeros_like(feat[nmax:]), atol=1e-8)

### Sanity check 2: Coefficients should be (for pyLODE: mostly) independent of the cell

In [ ]:
# For librascal and rascaline, the features should be completely independent of the cell
for feats in [features_librascal_single]:
    feat1 = feats[0]
    feat2 = feats[1]
    assert_allclose(feat1, feat2, rtol=1e-10)

In [ ]:
# For pyLODE, the k-space sum does lead to a weak dependence on the cell that depends on
# how well the k-space converged.
feat1 = features_pylode_single[0]
feat2 = features_pylode_single[1]
assert_allclose(feat1, feat2, rtol=1e-10, atol=3e-6)

### Compare pyLODE and rascal coefficients with exact values

In [ ]:
feat_pylode_l0_zero = features_pylode_single[0,0,:,0]

In [ ]:
coeffs_exact_center = coeffs_exact[:,0] # take the r_{ij} = 0 part

In [ ]:
print('Exact center contributions', coeffs_exact_center)

In [ ]:
print('Comparison to pyLODE')
print(feat_pylode_l0_zero / coeffs_exact_center)

In [ ]:
print('Comparison to librascal')
feat_librascal_l0_nonzeropart = features_librascal_single[0,::(lmax+1)**2]
print(feat_librascal_l0_nonzeropart / coeffs_exact_center)

In [ ]:
feat_librascal_l0_nonzeropart / feat_pylode_l0_zero

In [ ]:
1./(2*np.pi*smearing**2)**1.5

In [ ]:
feat_librascal_l0_nonzeropart

In [ ]:
print('Comparison to rascaline')
feat_rascaline_l0_nonzeropart = features_rascaline_single[0,:nmax]
print(feat_rascaline_l0_nonzeropart / coeffs_exact_center)

## Compare exact coefficients with those from pyLODE and Rascal: Dimers <a class="anchor" id="l=0dimer"></a>

TODO

## Norm tests

In [ ]:
features_pylode_single = calculator_pylode.features[-2:]
features_pylode_dimers = calculator_pylode.features[:-2]

In [ ]:
norm_exact_single = 1./(4 * np.pi * smearing**2)**0.75
norm_exact_dimers = np.zeros((Ndimers,))

for i, d in enumerate(distances):
    norm_exact_dimers[i] = norm_exact_single * np.sqrt(2 + np.exp(-d**2/(4 * smearing**2)))

In [ ]:
features_pylode_dimers.shape

In [ ]:
for features in features_pylode_single:
    print(norm(features.flatten()))
    print(norm_exact_single)

In [ ]:
norm_pylode_single = np.zeros((2,))
for i in range(2):
    norm_pylode_single[i] = norm(features_pylode_single[i].flatten())

In [ ]:
print(norm_pylode_single)
print(norm_exact_single)

In [ ]:
norm_pylode_dimers = np.zeros_like(distances)
for i,d in enumerate(distances):
    norm_pylode_dimers[i] = norm(features_pylode_dimers[i].flatten())

In [ ]:
plt.scatter(distances, norm_exact_dimers, label='exact')
plt.scatter(distances, norm_pylode_dimers, label='pyLODE')
plt.legend()
plt.ylim(0,0.8)

# Compare coefficients for random molecules <a class="anchor" id="randommolecule"></a>

In [ ]:
frames = []
frames.append(Atoms('O4', positions=[[1,1,1],[2,1,1],[2,2.2,1],[2.3,2.,1.5]], cell=cell, pbc=True))
frames.append(Atoms('O4', positions=[[1,1,1],[2.3,1,1],[2.2,2.5,1.1],[2.7,2.,1.1]], cell=cell, pbc=True))

In [ ]:
features_librascal_single = calculator_librascal.transform(frames).get_features(calculator_librascal)
calculator_pylode.transform(frames)
features_pylode_single = calculator_pylode.features

In [ ]:
features_librascal_single.shape

In [ ]:
features_pylode_single = features_pylode_single[:,0,:,:]

In [ ]:
f_librascal = features_librascal_single.reshape((8, nmax, (lmax+1)**2))

In [ ]:
for i in range(8):
    ratiomax = np.max(features_pylode_single[i] / f_librascal[i] /2.82841)
    ratiomin = np.min(features_pylode_single[i] / f_librascal[i] /2.82841)
    print(ratiomax)
    print(ratiomin)

In [ ]:
np.max(np.abs(features_pylode_single[i] / f_librascal[i])/2.82841)

In [ ]:
features_pylode_single.shape

Comment: Coefficients now also agree for "generic" molecules (i.e. molecules that do not have any special symmetry) up to a global factor!

## Table of Contents:
* [Preparation](#preparation)
    * [Exact Expression for Coefficients](#exactexpressions)
    * [Get the features for dimers](#getfeatures)
* [Coefficients for $l=1$](#l=1)
    * [Explicit computation of coefficients for $l=1$](#l=1explicit)
    * [Comparison to pyLODE/rascal for dimers](#l=1dimer)
* [Coefficients for $l\geq 2$](#lgeq2)
    * [Coefficients for $l=2$](#l=2)
    * [Coefficients for general $l$](#l_general)
* [Coefficients for l=0](#l=0)
    * [Exact coefficients](#l=0exact)
    * [Comparison to pyLODE/rascal for center contribution](#l=0center)
    * [Comparison to pyLODE/rascal for dimers](#l=0dimer)